In [ ]:
import catboost
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# fake dataset 
X = np.random.random((100000, 100)) - 0.5
w = np.random.random((100,))
y = np.sign(X.dot(w))
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

# catboost pools
train_pool = catboost.Pool(X_tr, y_tr)
test_pool = catboost.Pool(X_te, y_te)

clf = catboost.CatBoostClassifier(
    train_dir='catboost_model_1',
    loss_function='Logloss',
    task_type='GPU'
)

clf.fit(
    train_pool,
    eval_set=test_pool,
    verbose=100,
    # plot=True  # doesn't work anyway
)

# visualizer doesn't work
# w = catboost.MetricVisualizer('catboost_model_1')
# w.start()